# MODELCHAIN

In [1]:
import pandas as pd
import pvlib
from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

In [3]:
temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

In [4]:
cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

cec_module = cec_modules['Canadian_Solar_Inc__CS6X_320P']
cec_inverter = cec_inverters['INGETEAM_POWER_TECHNOLOGY_S_A___Ingecon_Sun_40TL_U_M__480V_']

In [5]:
cec_module

Technology           Multi-c-Si
Bifacial                      0
STC                     319.792
PTC                       292.8
A_c                       1.901
Length                    1.948
Width                     0.976
N_s                          72
I_sc_ref                   9.26
V_oc_ref                   45.3
I_mp_ref                   8.69
V_mp_ref                   36.8
alpha_sc               0.004389
beta_oc               -0.139116
T_NOCT                     45.8
a_ref                   1.78528
I_L_ref                  9.2705
I_o_ref             8.72084e-11
R_s                    0.362235
R_sh_ref                319.379
Adjust                  5.85738
gamma_r                 -0.4002
BIPV                          N
Version       SAM 2018.11.11 r2
Date                   1/3/2019
Name: Canadian_Solar_Inc__CS6X_320P, dtype: object

In [7]:
cec_inverter

Vac                          480
Pso                       75.262
Paco                       39950
Pdco                     41044.5
Vdco                         700
C0                  -4.21213e-07
C1                      -3.3e-05
C2                      0.000303
C3                     -0.001313
Pnt                       11.985
Vdcmax                       820
Idcmax                   58.6349
Mppt_low                     520
Mppt_high                    820
CEC_Date               5/15/2018
CEC_Type     Utility Interactive
Name: INGETEAM_POWER_TECHNOLOGY_S_A___Ingecon_Sun_40TL_U_M__480V_, dtype: object

In [8]:
latitude= 20.56
longitude= -103.22
altitude=1544
tz='America/Mexico_City'

In [9]:
location = Location(latitude=latitude, longitude=longitude, altitude=altitude, tz=tz)

In [10]:
surface_tilt = 25
surface_azimuth = 180 # pvlib uses 0=North, 90=East, 180=South, 270=West convention
albedo = 0.2

In [11]:
system = PVSystem(surface_tilt=surface_tilt, surface_azimuth=surface_azimuth,
                 module_parameters=cec_module,
                 inverter_parameters=cec_inverter,
                 temperature_model_parameters=temperature_model_parameters,
                 albedo=albedo, surface_type=None, module=None, 
                 module_type='glass_polymer',
                 modules_per_string=19.5, 
                 strings_per_inverter=8, inverter=None, 
                 racking_model='open_rack', 
                 losses_parameters=None, 
                 name='Name of PV system: Huerto de Cutonalá')

In [12]:
#system

In [13]:
mc = ModelChain(system, location, name='Huerto de Cutonalá', 
                clearsky_model='ineichen', transposition_model='haydavies', 
                solar_position_method='nrel_numpy', airmass_model='kastenyoung1989',
                dc_model=None, #from module specifications
                ac_model='sandia', #from inverter specifications
                aoi_model='no_loss', 
                spectral_model='no_loss', #spectral model 'no loss' otherwise error
                temperature_model='sapm', losses_model='no_loss')

In [14]:
#mc

In [15]:
import datetime
#Variables de tiempo

start = pd.Timestamp(datetime.date.today(), tz='America/Mexico_City') # today's date
end = start + pd.Timedelta(days=7) # 7 days from today

In [16]:
from pvlib import solarposition, irradiance, atmosphere, pvsystem, inverter, temperature, iam
from pvlib.forecast import GFS, NAM, NDFD, RAP, HRRR

/Users/luisvargas/Desktop/Tesis Pvlib/env_pvlib_tesis/lib/python3.8/site-packages/pvlib/forecast.py:18: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  warnings.warn(


In [36]:
# Se define el modelo de pronostico
fm = GFS()
#fm = NAM()
#fm = NDFD()
#fm = RAP()
#fm = HRRR()

In [37]:
# Se obtiene la data y se procesa del modelo de pronostico
forecast_data = fm.get_processed_data(latitude, longitude, start, end)

In [38]:
#pd.set_option('display.max_columns', None)
forecast_data.head()

,temp_air,wind_speed,ghi,dni,dhi,total_clouds,low_clouds,mid_clouds,high_clouds
2020-12-09 06:00:00-06:00,14.222504,0.530006,0.000000,0.000000,0.000000,96.0,0.0,38.0,94.0
2020-12-09 09:00:00-06:00,13.035370,0.824129,178.799104,107.607499,142.093117,60.0,0.0,1.0,60.0
2020-12-09 12:00:00-06:00,11.983917,1.518421,526.032816,260.404947,341.456002,43.0,0.0,1.0,43.0
2020-12-09 15:00:00-06:00,18.998260,1.151956,577.015548,813.945280,103.736876,0.0,0.0,0.0,0.0
2020-12-09 18:00:00-06:00,32.651489,0.900248,4.974719,0.000000,4.974719,1.0,0.0,1.0,0.0


In [39]:
ghi = forecast_data['ghi']
dni = forecast_data['dni']
dhi = forecast_data['dhi']
temp_air = forecast_data['temp_air']
wind_speed = forecast_data['wind_speed']
indexDF = forecast_data.index

In [40]:
weather = pd.DataFrame([dni, ghi, dhi, temp_air, wind_speed]).T

In [41]:
mc.run_model(weather)

ModelChain: 
  name: Huerto de Cutonalá
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [42]:
mc.dc

,i_sc,v_oc,i_mp,v_mp,p_mp,i_x,i_xx
2020-12-09 06:00:00-06:00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-12-09 09:00:00-06:00,1.493657e+01,844.110553,1.408174e+01,725.886156,10221.740241,1.482703e+01,1.124183e+01
2020-12-09 12:00:00-06:00,4.423736e+01,848.585170,4.151528e+01,705.699374,29297.304984,4.391226e+01,3.104365e+01
2020-12-09 15:00:00-06:00,5.800239e+01,819.575619,5.414254e+01,665.011384,36005.404384,5.759063e+01,3.920319e+01
2020-12-09 18:00:00-06:00,3.563520e-01,670.195684,3.321559e-01,568.651018,188.880779,3.542609e-01,2.669438e-01
2020-12-09 21:00:00-06:00,4.135903e-25,0.000000,4.135903e-25,0.000000,0.000000,4.135903e-25,4.135903e-25
2020-12-10 00:00:00-06:00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-12-10 03:00:00-06:00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-12-10 06:00:00-06:00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-12-10 09:00:00-06:00,3.239770e+01,850.297554,3.046970e+01,717.033045,21847.782210,3.215891e+01,2.338499e+01


In [43]:
mc.ac

2020-12-09 06:00:00-06:00      -11.985000
2020-12-09 09:00:00-06:00    10029.205032
2020-12-09 12:00:00-06:00    28643.829378
2020-12-09 15:00:00-06:00    35076.433846
2020-12-09 18:00:00-06:00      115.576301
2020-12-09 21:00:00-06:00      -11.985000
2020-12-10 00:00:00-06:00      -11.985000
2020-12-10 03:00:00-06:00      -11.985000
2020-12-10 06:00:00-06:00      -11.985000
2020-12-10 09:00:00-06:00    21414.568588
2020-12-10 12:00:00-06:00    39950.000000
2020-12-10 15:00:00-06:00    34960.819692
2020-12-10 18:00:00-06:00      132.942580
2020-12-10 21:00:00-06:00      -11.985000
2020-12-11 00:00:00-06:00      -11.985000
2020-12-11 03:00:00-06:00      -11.985000
2020-12-11 06:00:00-06:00      -11.985000
2020-12-11 09:00:00-06:00    21342.405110
2020-12-11 12:00:00-06:00    39950.000000
2020-12-11 15:00:00-06:00    35183.321543
2020-12-11 18:00:00-06:00      151.132398
2020-12-11 21:00:00-06:00      -11.985000
2020-12-12 00:00:00-06:00      -11.985000
2020-12-12 03:00:00-06:00      -11

In [44]:
mc.airmass

,airmass_relative,airmass_absolute
2020-12-09 06:00:00-06:00,NaN,NaN
2020-12-09 09:00:00-06:00,2.905578,2.411658
2020-12-09 12:00:00-06:00,1.408879,1.169383
2020-12-09 15:00:00-06:00,1.715667,1.424020
2020-12-09 18:00:00-06:00,18.656705,15.485243
2020-12-09 21:00:00-06:00,NaN,NaN
2020-12-10 00:00:00-06:00,NaN,NaN
2020-12-10 03:00:00-06:00,NaN,NaN
2020-12-10 06:00:00-06:00,NaN,NaN
2020-12-10 09:00:00-06:00,2.924307,2.427203


In [108]:
mc.cec

<bound method ModelChain.cec of ModelChain: 
  name: Huerto de Cutonalá
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses>

In [109]:
mc.cell_temperature

2020-12-08 06:00:00-06:00    13.304749
2020-12-08 09:00:00-06:00    14.849089
2020-12-08 12:00:00-06:00    19.365076
2020-12-08 15:00:00-06:00    25.877347
2020-12-08 18:00:00-06:00    31.285581
2020-12-08 21:00:00-06:00    29.402191
2020-12-09 00:00:00-06:00    18.017914
2020-12-09 03:00:00-06:00    15.238037
2020-12-09 06:00:00-06:00    14.003265
2020-12-09 09:00:00-06:00    16.908342
2020-12-09 12:00:00-06:00    24.315839
2020-12-09 15:00:00-06:00    23.688893
2020-12-09 18:00:00-06:00    32.185004
2020-12-09 21:00:00-06:00    31.409882
2020-12-10 00:00:00-06:00    18.912842
2020-12-10 03:00:00-06:00    15.192535
2020-12-10 06:00:00-06:00    13.458618
2020-12-10 09:00:00-06:00    26.588806
2020-12-10 12:00:00-06:00    43.464736
2020-12-10 15:00:00-06:00    44.744162
2020-12-10 18:00:00-06:00    30.552687
2020-12-10 21:00:00-06:00    29.598419
2020-12-11 00:00:00-06:00    18.973175
2020-12-11 03:00:00-06:00    14.991577
2020-12-11 06:00:00-06:00    13.718842
2020-12-11 09:00:00-06:00

In [110]:
mc.solar_position

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2020-12-08 06:00:00-06:00,107.715367,107.715367,-17.715367,-17.715367,108.327857,7.931504
2020-12-08 09:00:00-06:00,69.879035,69.916706,20.120965,20.083294,125.298173,7.875987
2020-12-08 12:00:00-06:00,44.730395,44.744260,45.269605,45.255740,165.177690,7.820347
2020-12-08 15:00:00-06:00,54.421896,54.440999,35.578104,35.559001,218.974696,7.764585
2020-12-08 18:00:00-06:00,87.903717,88.128916,2.096283,1.871084,244.722970,7.708702
2020-12-08 21:00:00-06:00,127.954555,127.954555,-37.954555,-37.954555,256.503283,7.652699
2020-12-09 00:00:00-06:00,169.236595,169.236595,-79.236595,-79.236595,255.604062,7.596576
2020-12-09 03:00:00-06:00,148.707289,148.707289,-58.707289,-58.707289,100.493823,7.540335
2020-12-09 06:00:00-06:00,107.840226,107.840226,-17.840226,-17.840226,108.396286,7.483978
2020-12-09 09:00:00-06:00,70.017722,70.055489,19.982278,19.944511,125.320525,7.427503


In [111]:
mc.ac_model

<bound method ModelChain.snlinverter of ModelChain: 
  name: Huerto de Cutonalá
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses>